# 敵対的バンディット

**参考**

* [Bandit Algorithms](https://tor-lattimore.com/downloads/book/book.pdf)

前回は[UCBアルゴリズムの証明](BANDIT_UCB_regret_proof.ipynb)をやりました。
今回は敵対的バンディットのアルゴリズムであるEXP3アルゴリズムの証明をみてみます。

**表記**

* $k > 1$：アームの数
* $(x_t)_{t=1}^n$：時刻$1, \dots, n$の報酬ベクトル
    * $x_t \in [0, 1]^k$：報酬のベクトル
    * 敵対者は前もって$(x_t)_{t=1}^n$を選択しておく
* $P_t \in \mathcal{P}_{k-1}$：学習者が行動を選ぶ分布

**目的関数**

次の期待リグレットと最悪ケースのリグレットを考えます。

* 期待リグレット：$R_n(\pi, x)=\max _{i \in[k]} \sum_{t=1}^n x_{t i}-\mathbb{E}\left[\sum_{t=1}^n x_{t A_t}\right]$
* 最悪リグレット：$R_n^*(\pi)=\sup _{x \in[0,1]^{n \times k}} R_n(\pi, x)$

この最悪リグレットを劣線形にするのが敵対的バンディットの目的です。
ここで、学習者が決定的な方策を選ぶケースを考えてみましょう。

## 敵対的バンディットにおける決定的方策のリグレット

学習者が決定的な方策を選択するとき、$R_n(\pi, x) \geq n(1-1 / k)$となるような$x$が必ず存在します。

証明しましょう。決定的な方策のとき、$A_t$は$x_{1, A_1}, \ldots, x_{t-1, A_{t-1}}$の関数です。
ここで一旦、決定的な方策の場合の問題設定の流れを時刻１から順番に見てみましょう。
1. 学習者は行動$A_1$を選択します。報酬$x_{1A_1}$を得ます
2. $A_1, x_{1 A_1}$を元に、学習者は行動$A_2$を選択します。報酬$x_{2 A_2}$を得ます
3. $A_1, x_{1 A_1}, A_2, x_{2 A_2}$を元に、学習者は行動$A_3$を選択します。報酬$x_{3 A_3}$を得ます
4. ...

これをよく見てみましょう。
行動が決定的であれば、$A_1$は始まる前から決定しています。
また、$x_{1 A_1}$も始まる前から決定できるので、$A_2$も始まる前から決定しています。
これを順番に繰り返すと、学習者が取る行動と真反対の報酬系列$x_{1, A_1}, \ldots, x_{t-1, A_{t-1}}$を設定することができます。つまり、

$$x_{t i}= \begin{cases}0 & \text { if } A_t=i \\ 1 & \text { otherwise }\end{cases}$$

とすれば、学習者は常に報酬０を集めることになります。
このときリグレットは
$R_n(\pi, x)=\max _{i \in[k]} \sum_{t=1}^n x_{t i}$
になります。
一方で、
$\max _{i \in[k]} \sum_{t=1}^n x_{t i} \geq \frac{1}{k} \sum_{t=1}^n \sum_{i=1}^k x_{t i}=\frac{n(k-1)}{k}$は学習者に関係なく常に成立するので、

$$
\max _{i \in[k]} \sum_{t=1}^n x_{t i} \geq \frac{1}{k} \sum_{t=1}^n \sum_{i=1}^k x_{t i}=\frac{n(k-1)}{k}
$$
が成り立ちます。

よって、決定的な方策を考える限りは敵対的バンディットで劣線形なリグレットの達成は不可能ですね。
以降は確率的な方策を考えていきます。


## リグレットの下界

敵対的バンディットではどこまでリグレットを小さくできるのでしょうか？
実は、確率的バンディットで敵対的バンディットのリグレットを下から抑えることができます。
簡単なイェンセン不等式から、

$$
\begin{aligned}
R_n(\pi, \nu) & =\max _{i \in[k]} \mathbb{E}\left[\sum_{t=1}^n\left(X_{t i}-X_{t A_t}\right)\right] \\
& \leq \mathbb{E}\left[\max _{i \in[k]} \sum_{t=1}^n\left(X_{t i}-X_{t A_t}\right)\right] \\
& =\mathbb{E}\left[R_n(\pi, X)\right] \leq R_n^*(\pi),
\end{aligned}
$$

が成り立ちます。また、確率的バンディットの下界は$c\sqrt{nk}$で抑えられるので（TODO）、
$$R_n^*=\inf _\pi \sup _{x \in[0,1]^{n \times k}} R_n(\pi, x) \geq c \sqrt{n k}$$
が成り立ちます。

## 報酬関数の推定

敵対的バンディットでは推定した報酬値をアルゴリズムで使います。
ここで、$P_{t i}=\mathbb{P}\left(A_t=i \mid A_1, X_1, \ldots, A_{t-1}, X_{t-1}\right)$を過去の履歴をもとにした$A_t$の条件付き確率とします。つまり時刻$t$で引くアームの分布ですね。
以下では方策が確率的であり、$P_{ti}>0$がalmost surelyで成り立っているとします。

時刻$t$のアーム$i$の報酬$x_{ti}$の推定値を

$$
\hat{X}_{t i}=\frac{\mathbb{I}\left\{A_t=i\right\} X_t}{P_{t i}}
$$

として推定していきましょう。
また、報酬の推定値の総和を
$$\hat{S}_{t i}=\sum_{s=1}^t \hat{X}_{s i}$$
とします。

ここで次が成り立ちます。

---

**報酬関数の普遍推定**

$\mathbb{E}_t[\cdot]=\mathbb{E}\left[\cdot \mid A_1, X_1, \ldots, A_t, X_t\right]$とすれば、$\mathbb{E}_{t-1}\left[\hat{X}_{t i}\right]=x_{t i}$が成り立ちます。
つまり、$t-1$までの履歴を踏まえた上では、時刻$t$の推定値は普遍推定です。

証明しましょう。次の表記を導入します。

* $A_{t i}=\mathbb{I}\left\{A_t=i\right\}$：時刻$t$でアーム$i$を引いていると１です。
    * $X_t A_{t i}=x_{t i} A_{t i}$：$X_t$は時刻とアームに対して決定的なので成り立ちます。
    * $\hat{X}_{t i}=\frac{A_{t i}}{P_{t i}} x_{t i}$

ここで、$\mathbb{E}_{t-1}\left[A_{t i}\right]=P_{t i}$かつ$P_{t i}$は$\sigma\left(A_1, X_1, \ldots, A_{t-1}, X_{t-1}\right)$加測であることを踏まえると、

$$
\mathbb{E}_{t-1}\left[\hat{X}_{t i}\right]=\mathbb{E}_{t-1}\left[\frac{A_{t i}}{P_{t i}} x_{t i}\right]=\frac{x_{t i}}{P_{t i}} \mathbb{E}_{t-1}\left[A_{t i}\right]=\frac{x_{t i}}{P_{t i}} P_{t i}=x_{t i}
$$
が成り立っています。

---

---

**報酬の総和の普遍推定**

$
\mathbb{E}\left[\hat{S}_{n i}\right]=\sum_{t=1}^n x_{t i}
$が成り立ちます。

証明しましょう。

$$
\mathbb{E}\left[\hat{S}_{n i}\right]=
\mathbb{E}\left[\sum_{t=1}^n \hat{X}_{t i}\right]
= \mathbb{E}\left[\sum_{t=1}^n \mathbb{E}_{t-1}\left[\hat{X}_{t i}\right]\right]
= \mathbb{E}\left[\sum_{t=1}^n x_{t i}\right]
$$

２つ目の等式はタワールールで成立します。

---

推定値が普遍推定なのも大事ですが、推定値のバリアンスも同じくらい大事な要素です。
ここで、適当な確率変数$U$について、その分散を
$$
\mathbb{V}_{t-1}[U]=\mathbb{E}_{t-1}\left[\left(U-\mathbb{E}_{t-1}[U]\right)^2\right]
$$
とします。
すると、普遍推定の式を変形すれば

$$
\mathbb{V}_{t-1}\left[\hat{X}_{t i}\right]=\mathbb{E}_{t-1}\left[\hat{X}_{t i}^2\right]-x_{t i}^2=\mathbb{E}_{t-1}\left[\frac{A_{t i} x_{t i}^2}{P_{t i}^2}\right]-x_{t i}^2=\frac{x_{t i}^2\left(1-P_{t i}\right)}{P_{t i}}
$$

が分散について成り立っていることがわかります。ここで、$P_{t i}$が非常に小さく、$x_{t i}$が大きい場合、分散がかなり大きくなってしまうことがわかります。

どうすればいいでしょうか？ここで、別の普遍推定を導入してみましょう。

$$
\hat{X}_{t i}=1-\frac{\mathbb{I}\left\{A_t=i\right\}}{P_{t i}}\left(1-X_t\right)
$$

これも$x_{t i}$の普遍推定です。
$y_{t i}=1 - x_{t i}$、$Y_t = 1 - X_t$、$\hat{Y}_{t i} = 1 - \hat{X}_{t i}$とすれば、

$$
\hat{Y}_{t i}=\frac{\mathbb{I}\left\{A_t=i\right\}}{P_{t i}} Y_t
$$

なる推定値が得られ、これも$y_{t i}=1 - x_{t i}$についての普遍推定であることがわかります。
これは**Lossベースの推定値**と呼ばれます。このバリアンスは
$$
\mathbb{V}_t\left[\hat{X}_{t i}\right]=\mathbb{V}_t\left[\hat{Y}_{t i}\right]=y_{t i}^2 \frac{1-P_{t i}}{P_{t i}}
$$
になっています。まとめると、

|                              Estimator                             |                      Variance                     |
|:------------------------------------------------------------------:|:-------------------------------------------------:|
| $\hat{X}_{t i}=\frac{\mathbb{I}\left\{A_t=i\right\} X_t}{P_{t i}}$ | $x_{t i}^2\frac{1-P_{t i}}{P_{t i}}$ | 
| $\hat{Y}_{t i}=\frac{\mathbb{I}\left\{A_t=i\right\} Y_t}{P_{t i}}$ |       $y_{t i}^2 \frac{1-P_{t i}}{P_{t i}}$       | 

なので、適宜$X$と$Y$の推定は入れ替えると推定値の質がよくなります。

## EXP3アルゴリズム

**表記**

* 学習率：$\eta > 0$

敵対的バンディットを解くための最も基本的なアルゴリズムがEXP3アルゴリズムです。
EXP3アルゴリズムは次のように進みます

1. Input: $n, k, \eta$
2. Set $\hat{S}_{0 i}=0$ for all $i$
3. for $t=1, \ldots, n$ do
    1. Calculate the sampling distribution $P_t:$
    $$
    P_{t i}=\frac{\exp \left(\eta \hat{S}_{t-1, i}\right)}{\sum_{j=1}^k \exp \left(\eta \hat{S}_{t-1, j}\right)}
    $$
    2. Sample $A_t \sim P_t$ and observe reward $X_t$
    3. Calculate $\hat{S}_{t i}$ :
    $$
    \hat{S}_{t i}=\hat{S}_{t-1, i}+1-\frac{\mathbb{I}\left\{A_t=i\right\}\left(1-X_t\right)}{P_{t i}}
    $$
4. end for

ここで推定値はLossベースの推定値が使われていることに注意しましょう。

このリグレットバウンドを導出しましょう。

### EXP3のリグレット解析

敵対的バンディットでは、[確率的バンディット](BANDIT_utils.ipynb)のように各アームに平均値が設定されているわけではないことに注意しましょう。
なので各アームの推定値の離れ具合をAzuma-Hoeffidingでバウンドする戦法を取るのは微妙そうですね。
敵対的バンディットのリグレットバウンドでは違う戦法（$\exp$をバウンドする？）を取ります。

敵対的バンディットでは、
$$R_{n i}=\sum_{t=1}^n x_{t i}-\mathbb{E}\left[\sum_{t=1}^n X_t\right]$$
をアーム$i$についてのリグレットとして、全てのアーム$i$についてのリグレットを導出していきます
（UCBでは$\sum_{t=0}^{T-1} \mu_{a^{\star}}-\mu_{A_t}$のうち、$\mu_{a^*}$を楽観的な$\mu$で上から抑えていました）。

まず、報酬が普遍推定であることを踏まえると次が成り立ちます：

* $\mathbb{E}\left[\hat{S}_{n i}\right]=\sum_{t=1}^n x_{t i}$：上でやりました
* $\mathbb{E}_{t-1}\left[X_t\right]=\sum_{i=1}^k P_{t i} x_{t i}=\sum_{i=1}^k P_{t i} \mathbb{E}_{t-1}\left[\hat{X}_{t i}\right]$

また、タワールールを同じように適用すれば

$$
R_{n i}
=\mathbb{E}\left[\hat{S}_{n i}\right]-\mathbb{E}\left[\sum_{t=1}^n \sum_{i=1}^k P_{t i} \hat{X}_{t i}\right]
=\mathbb{E}\left[\hat{S}_{n i}-\hat{S}_n\right]
$$
が成り立ちます。ここで$\hat{S}_n=\sum_t \sum_i P_{t i} \hat{X}_{t i}$と定義しています。

この右辺をバウンドすれば良いわけですね。
まず、$\exp \left(\eta \hat{S}_{n i}\right) \leq \sum_{j=1}^k \exp \left(\eta \hat{S}_{n j}\right)$が明らかに成り立っていることに注意します。
これを$W_t=\sum_{j=1}^k \exp \left(\eta \hat{S}_{t j}\right)$とします。

すると、

$$
\exp \left(\eta \hat{S}_{n i}\right) \leq \sum_{j=1}^k \exp \left(\eta \hat{S}_{n j}\right)=W_n=W_0 \frac{W_1}{W_0} \cdots \frac{W_n}{W_{n-1}}=k \prod_{t=1}^n \frac{W_t}{W_{t-1}}
$$

が成り立ちます（empty sumを$0$として、$W_0=k$としました。）

このとき、$P_{t j}$の定義から、
$$
\frac{W_t}{W_{t-1}}=\sum_{j=1}^k \frac{\exp \left(\eta \hat{S}_{t-1, j}\right)}{W_{t-1}} \exp \left(\eta \hat{X}_{t j}\right)=\sum_{j=1}^k P_{t j} \exp \left(\eta \hat{X}_{t j}\right)
$$

が成り立ちます。
ここで、
* $\exp (x) \leq 1+x+x^2$ for all $x \leq 1 \quad$ 
* $\quad 1+x \leq \exp (x)$ for all $x \in \mathbb{R}$

が成立することに注目しましょう。今、**推定値をLossベースにしているおかげ**で１つ目の不等式が使えます。
これを使うと、

$$
\begin{aligned}
\frac{W_t}{W_{t-1}} & \leq 1+\eta \sum_{j=1}^k P_{t j} \hat{X}_{t j}+\eta^2 \sum_{j=1}^k P_{t j} \hat{X}_{t j}^2 \\
& \leq \exp \left(\eta \sum_{j=1}^k P_{t j} \hat{X}_{t j}+\eta^2 \sum_{j=1}^k P_{t j} \hat{X}_{t j}^2\right)
\end{aligned}
$$

が成り立ちます。

最終的に、
$$
\exp \left(\eta \hat{S}_{n i}\right) \leq k \exp \left(\eta \hat{S}_n+\eta^2 \sum_{t=1}^n \sum_{j=1}^k P_{t j} \hat{X}_{t j}^2\right)
$$
が成立します。
両辺に$\log$を取って、

$$
\hat{S}_{n i}-\hat{S}_n \leq \frac{\log (k)}{\eta}+\eta \sum_{t=1}^n \sum_{j=1}^k P_{t j} \hat{X}_{t j}^2
$$

が成立します。あとは二項目をバウンドするだけです。このために新しい表記を導入します。
* $y_{t j}=1-x_{t j}$ 
* $Y_t=1-X_t$

これを使って、

$$
\begin{aligned}
\mathbb{E}\left[\sum_{j=1}^k P_{t j} \hat{X}_{t j}^2\right] & =\mathbb{E}\left[\sum_{j=1}^k P_{t j}\left(1-\frac{\mathbb{I}\left\{A_t=j\right\} y_{t j}}{P_{t j}}\right)^2\right] \\
& =\mathbb{E}\left[\sum_{j=1}^k P_{t j}\left(1-2 \frac{\mathbb{I}\left\{A_t=j\right\} y_{t j}}{P_{t j}}+\frac{\mathbb{I}\left\{A_t=j\right\} y_{t j}^2}{P_{t j}^2}\right)\right] \\
& =\mathbb{E}\left[1-2 Y_t+\mathbb{E}_{t-1}\left[\sum_{j=1}^k \frac{\mathbb{I}\left\{A_t=j\right\} y_{t j}^2}{P_{t j}}\right]\right] \\
& =\mathbb{E}\left[1-2 Y_t+\sum_{j=1}^k y_{t j}^2\right] \\
& =\mathbb{E}\left[\left(1-Y_t\right)^2+\sum_{j \neq A_t} y_{t j}^2\right] \\
& \leq k .
\end{aligned}
$$

最後の不等式は$y_{t j} \leq 1$であることを使えば明らかに成り立ちます。

よって、
$$
R_{n i} \leq \frac{\log (k)}{\eta}+\eta n k
$$

が成り立ちます。$\eta$は自由に選んでいいので、$\eta=\sqrt{\log (k) /(n k)}$とすれば、
$$
R_{n i} \leq 2 \sqrt{n k \log (k)}
$$
が成り立ちます。

下界が
$$
R_{n i} \geq c \sqrt{n k}
$$
だったことを考えると、これはほぼMinimax最適になっていますね。

実装は[BANDIT_utils.ipynb](BANDIT_utils.ipynb)に任せます。
